In [5]:
import pandas as pd
import gspread
from datetime import datetime
from gspread_dataframe import set_with_dataframe
from functools import reduce
import yfinance as yf
pd.set_option('display.max_row', 1500)


In [7]:
#Extracting actual companies in S&P 500 Index + calculating lenght of companies in the index
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = table[0]
sp500 = sp500["Symbol"].tolist()
sp500_len = len(sp500)

In [10]:
list_stocks=[]
for i in range(0,sp500_len):
    stock = yf.Ticker(sp500[i])
    stock_i=stock.info
    stock_i=pd.DataFrame.from_dict(stock_i.items())
    stock_i.columns =['Index', 'Value'] 
    stock_i.rename(columns = {'Value':sp500[i]}, inplace = True)
    list_stocks.append(stock_i)

final_df = reduce(lambda  left,right: pd.merge(left,right,on=['Index'], how='outer'), list_stocks)
final_df = final_df.reset_index(drop=True).T
final_df.columns=final_df.iloc[0]
final_df=final_df.iloc[1:, :]
new_df = final_df[['sector']]
new_df = new_df.rename(columns={'sector': 'Sector'})
new_df = new_df.reset_index()
new_df = new_df.rename(columns={'index': 'Name'})

In [11]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = table[0]
sp500 = sp500["Symbol"].tolist()
sp500_len = len(sp500)
value_stock = yf.Ticker(sp500[0])
sp500_len = len(sp500)
income_statement = value_stock.income_stmt
income_statement = income_statement.reset_index().rename(columns={'index': 'Metrics'})
income_statement = pd.melt(income_statement, id_vars=['Metrics'], var_name='Date', value_name='Value')
income_statement['Stock'] = value_stock.info['symbol']
income_statement['Value'] = income_statement['Value'] / 1000000

In [17]:
income_statement_cobined = pd.DataFrame()

# Iterate through each company in the S&P 500
for symbol in sp500:
    value_stock = yf.Ticker(symbol)
    income_statement = value_stock.income_stmt
    income_statement = income_statement.reset_index().rename(columns={'index': 'Metrics'})
    income_statement = pd.melt(income_statement, id_vars=['Metrics'], var_name='Date', value_name='Value')
    income_statement['Name'] = value_stock.info['symbol']
    income_statement['Value'] = income_statement['Value'] / 1000000
        
    # Concatenate data for the current company to the combined DataFrame
    income_statement_cobined = pd.concat([income_statement_cobined, income_statement], ignore_index=True)
    
income_statement_cobined['Year'] = income_statement_cobined['Date'].dt.year
income_statement_cobined = income_statement_cobined.drop(columns=['Date'])

income_statement_cobined = pd.merge(new_df, income_statement_cobined, on='Name', how='left')
income_statement_cobined['Report'] = 'Income_Statement'


In [18]:
balance_sheet_cobined = pd.DataFrame()

# Iterate through each company in the S&P 500
for symbol in sp500:
    value_stock = yf.Ticker(symbol)
    balance_sheet = value_stock.balance_sheet
    balance_sheet = balance_sheet.reset_index().rename(columns={'index': 'Metrics'})
    balance_sheet = pd.melt(balance_sheet, id_vars=['Metrics'], var_name='Date', value_name='Value')
    balance_sheet['Name'] = value_stock.info['symbol']
    balance_sheet['Value'] = balance_sheet['Value'] / 1000000
        
    # Concatenate data for the current company to the combined DataFrame
    balance_sheet_cobined = pd.concat([balance_sheet_cobined, balance_sheet], ignore_index=True)

balance_sheet_cobined['Year'] = balance_sheet_cobined['Date'].dt.year
balance_sheet_cobined = balance_sheet_cobined.drop(columns=['Date'])

balance_sheet_cobined = pd.merge(new_df, balance_sheet_cobined, on='Name', how='left')
balance_sheet_cobined['Report'] = 'Balance_Sheet'


In [19]:
cashflow_cobined = pd.DataFrame()

# Iterate through each company in the S&P 500
for symbol in sp500:
    value_stock = yf.Ticker(symbol)
    cashflow = value_stock.cashflow
    cashflow = cashflow.reset_index().rename(columns={'index': 'Metrics'})
    cashflow = pd.melt(cashflow, id_vars=['Metrics'], var_name='Date', value_name='Value')
    cashflow['Name'] = value_stock.info['symbol']
    cashflow['Value'] = cashflow['Value'] / 1000000
        
    # Concatenate data for the current company to the combined DataFrame
    cashflow_cobined = pd.concat([cashflow_cobined, cashflow], ignore_index=True)

cashflow_cobined['Year'] = cashflow_cobined['Date'].dt.year
cashflow_cobined = cashflow_cobined.drop(columns=['Date'])

cashflow_cobined = pd.merge(new_df, cashflow_cobined, on='Name', how='left')
cashflow_cobined['Report'] = 'Cash_Flow'


In [23]:
tota_data = pd.DataFrame()
tota_data = pd.concat([balance_sheet_cobined, cashflow_cobined, income_statement_cobined], ignore_index=True)


In [22]:
GSHEET_NAME = 'S&P 500 Reports'
TAB_NAME = 'Data_SP500' 

def write_df_to_sheet(GSHEET_NAME, TAB_NAME, df):
    gc= gspread.service_account(filename ='sp500-reporting-848fbf6683ca.json')
    sh=gc.open(GSHEET_NAME)
    worksheet = sh.worksheet(TAB_NAME)
    set_with_dataframe(worksheet, df)

write_df_to_sheet(GSHEET_NAME, TAB_NAME, tota_data) 